In [2]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.externals import joblib
import time

import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

In [2]:
propiedades = pd.read_csv('/home/mati/Desktop/set_datos_propiedades.csv')

In [3]:
propiedades = propiedades.loc[(propiedades.price_aprox_usd.notnull()) & (propiedades.superficie.notnull()),\
                             ['place_name_encoded', 'property_type_encoded','price_aprox_usd','superficie',\
                             'Year','Month','seguridad','aire','gimnasio','cochera','pileta']]

In [4]:
propiedades.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1132495 entries, 0 to 1413024
Data columns (total 11 columns):
place_name_encoded       1132495 non-null int64
property_type_encoded    1132495 non-null int64
price_aprox_usd          1132495 non-null float64
superficie               1132495 non-null float64
Year                     1132495 non-null int64
Month                    1132495 non-null int64
seguridad                1132495 non-null bool
aire                     1132495 non-null bool
gimnasio                 1132495 non-null bool
cochera                  1132495 non-null bool
pileta                   1132495 non-null bool
dtypes: bool(5), float64(2), int64(4)
memory usage: 65.9 MB


# Decision Tree

In [14]:
columnas = ['superficie','place_name_encoded','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

In [15]:
set_entrenamiento = propiedades.loc[(propiedades.Year >= 2016) &((propiedades.Year < 2017) | (propiedades.Month < 6))\
                                    ,columnas_precio]
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),columnas_precio].head(20000)

set_entrenamiento_datos = set_entrenamiento.loc[:,columnas]
set_entrenamiento_resultado = set_entrenamiento.loc[:,'price_aprox_usd']

In [16]:
tree = DecisionTreeRegressor()
tree.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply(lambda x: tree.predict(x)[0],axis = 1)
precision = tree.score(set_pruebas.loc[:,columnas],set_pruebas.price_aprox_usd) * 100
error = mean_squared_error(set_pruebas.price_aprox_usd,set_pruebas.resultado)
print("Precision = {:.2f} % , error = {}".format(precision, error))

Precision = 90.99 % , error = 12434347138.596506


## Ahora que tenemos una intuicion, probamos cambiando los parametros

In [7]:
columnas = ['superficie','place_name_encoded','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

In [8]:
set_entrenamiento = propiedades.loc[(propiedades.Year >= 2016) &((propiedades.Year < 2017) | (propiedades.Month < 6))\
                                    ,columnas_precio]
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),columnas_precio].head(20000)

set_entrenamiento_datos = set_entrenamiento.loc[:,columnas]
set_entrenamiento_resultado = set_entrenamiento.loc[:,'price_aprox_usd']

res = []

In [9]:
lista_criterio = ['mse','friedman_mse']
lista_max_depth = [None,1,5,10]
lista_min_samples_split = [2,3]
lista_min_samples_leaf = [1,10]
lista_features = [0.4,0.6,1.0]

for criterio in lista_criterio:
    for max_depth in lista_max_depth:
        for min_sample in lista_min_samples_split:
            for min_leaf in lista_min_samples_leaf:
                for feature in lista_features:
                    
                    tree = DecisionTreeRegressor(criterion = criterio, max_depth = max_depth, min_samples_split = min_sample,\
                                    min_samples_leaf = min_leaf, max_features = feature)
                    tree.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
                    set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply\
                            (lambda x: tree.predict(x)[0],axis = 1)
                    precision = tree.score(set_pruebas.loc[:,columnas],set_pruebas.loc[:,'price_aprox_usd']) * 100
                    error = mean_squared_error(set_pruebas.price_aprox_usd,set_pruebas.resultado)
                    res.append((criterio,max_depth,min_sample,min_leaf,feature,precision,error))
                    print(criterio,' - ',max_depth,' - ',min_sample,' - ',min_leaf,' - ',feature)

mse  -  None  -  2  -  1  -  0.4
mse  -  None  -  2  -  1  -  0.6
mse  -  None  -  2  -  1  -  1.0
mse  -  None  -  2  -  10  -  0.4
mse  -  None  -  2  -  10  -  0.6
mse  -  None  -  2  -  10  -  1.0
mse  -  None  -  3  -  1  -  0.4
mse  -  None  -  3  -  1  -  0.6
mse  -  None  -  3  -  1  -  1.0
mse  -  None  -  3  -  10  -  0.4
mse  -  None  -  3  -  10  -  0.6
mse  -  None  -  3  -  10  -  1.0
mse  -  1  -  2  -  1  -  0.4
mse  -  1  -  2  -  1  -  0.6
mse  -  1  -  2  -  1  -  1.0
mse  -  1  -  2  -  10  -  0.4
mse  -  1  -  2  -  10  -  0.6
mse  -  1  -  2  -  10  -  1.0
mse  -  1  -  3  -  1  -  0.4
mse  -  1  -  3  -  1  -  0.6
mse  -  1  -  3  -  1  -  1.0
mse  -  1  -  3  -  10  -  0.4
mse  -  1  -  3  -  10  -  0.6
mse  -  1  -  3  -  10  -  1.0
mse  -  5  -  2  -  1  -  0.4
mse  -  5  -  2  -  1  -  0.6
mse  -  5  -  2  -  1  -  1.0
mse  -  5  -  2  -  10  -  0.4
mse  -  5  -  2  -  10  -  0.6
mse  -  5  -  2  -  10  -  1.0
mse  -  5  -  3  -  1  -  0.4
mse  -  5  -  3  - 

In [10]:
for r in res:
    print("Criterio = {}, max_depth = {}, min_sample_split = {}, min_sample_leaf = {}, features = {}, precision = {:.2f} % , error = {}"\
          .format(r[0],r[1],r[2],r[3],r[4],r[5],r[6]))

Criterio = mse, max_depth = None, min_sample_split = 2, min_sample_leaf = 1, features = 0.4, precision = 90.92 % , error = 12523178385.446316
Criterio = mse, max_depth = None, min_sample_split = 2, min_sample_leaf = 1, features = 0.6, precision = 90.84 % , error = 12633432697.540848
Criterio = mse, max_depth = None, min_sample_split = 2, min_sample_leaf = 1, features = 1.0, precision = 90.99 % , error = 12436424287.069954
Criterio = mse, max_depth = None, min_sample_split = 2, min_sample_leaf = 10, features = 0.4, precision = 61.57 % , error = 53024760632.939766
Criterio = mse, max_depth = None, min_sample_split = 2, min_sample_leaf = 10, features = 0.6, precision = 58.74 % , error = 56920716118.86883
Criterio = mse, max_depth = None, min_sample_split = 2, min_sample_leaf = 10, features = 1.0, precision = 62.36 % , error = 51921658182.3563
Criterio = mse, max_depth = None, min_sample_split = 3, min_sample_leaf = 1, features = 0.4, precision = 90.29 % , error = 13398537273.276394
Criter

In [11]:
min_error = float('inf')
max_precision = 0
tupla_min_error = ()
tupla_max_precision = ()
for r in res:
    if r[6] < min_error:
        min_error = r[6]
        tupla_min_error = r
    if r[5] > max_precision:
        max_precision = r[5]
        tupla_max_precision = r
        
print("Mayor precision = Criterio = {}, max_depth = {}, min_sample_split = {}, min_sample_leaf = {}, features = {}, precision = {:.2f} % , error = {}".\
              format(tupla_max_precision[0],tupla_max_precision[1],tupla_max_precision[2],tupla_max_precision[3],tupla_max_precision[4],tupla_max_precision[5],tupla_max_precision[6]))
print("Menor error = Criterio = {}, max_depth = {}, min_sample_split = {}, min_sample_leaf = {}, features = {}, precision = {:.2f} % , error = {}".\
              format(tupla_min_error[0],tupla_min_error[1],tupla_min_error[2],tupla_min_error[3],tupla_min_error[4],tupla_min_error[5],tupla_min_error[6]))

Mayor precision = Criterio = mse, max_depth = None, min_sample_split = 2, min_sample_leaf = 1, features = 1.0, precision = 90.99 % , error = 12436424287.069954
Menor error = Criterio = mse, max_depth = None, min_sample_split = 2, min_sample_leaf = 1, features = 1.0, precision = 90.99 % , error = 12436424287.069954


## Tomamos criterio = mse, max_depth = None ,min_sample = 2 ,min_leaf = 1, features = 1.0

In [56]:
#Probamos distintas combinaciones de columnas, esta es la que dio mejores resultados

columnas = ['superficie','place_name_encoded','property_type_encoded','seguridad','gimnasio','aire','pileta','cochera']
columnas_precio = columnas + ['price_aprox_usd']

In [57]:
set_entrenamiento = propiedades.loc[(propiedades.Year >= 2016) &((propiedades.Year < 2017) | (propiedades.Month < 6))\
                                    ,columnas_precio]
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),columnas_precio].head(20000)

set_entrenamiento_datos = set_entrenamiento.loc[:,columnas]
set_entrenamiento_resultado = set_entrenamiento.loc[:,'price_aprox_usd']

In [58]:
criterio = 'mse'
max_depth = None
min_sample = 2
min_leaf = 1
feature = 1.0

In [59]:
tree = DecisionTreeRegressor(criterion = criterio, max_depth = max_depth, min_samples_split = min_sample,\
                                    min_samples_leaf = min_leaf, max_features = feature)
tree.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply(lambda x: tree.predict(x)[0],axis = 1)
precision = tree.score(set_pruebas.loc[:,columnas],set_pruebas.price_aprox_usd) * 100
error = mean_squared_error(set_pruebas.price_aprox_usd,set_pruebas.resultado)
print("Precision = {:.2f} % , error = {}".format(precision, error))

Precision = 90.97 % , error = 12458466712.821684


# calculo con los verdaderos datos a analizar

In [60]:
analizar = pd.read_csv("/home/mati/Desktop/properati_dataset_modificado.csv")

In [62]:
analizar.loc[:,'price_usd'] = analizar.loc[:,columnas].apply(lambda x: tree.predict(x)[0],axis = 1)

In [63]:
resultado = analizar.loc[:,['id','price_usd']]

In [64]:
resultado.to_csv('Decision_tree_resultados', index = False)

# Ahora uso lat y lon en lugar de place name

In [2]:
propiedades = pd.read_csv('/home/mati/Desktop/set_datos_propiedades.csv')

In [3]:
propiedades = propiedades.loc[(propiedades.price_aprox_usd.notnull()) & (propiedades.superficie.notnull())\
                              & (propiedades.lat.notnull()) & (propiedades.lon.notnull()),\
                             ['lat', 'lon', 'property_type_encoded','price_aprox_usd','superficie',\
                             'Year','Month','seguridad','aire','gimnasio','cochera','pileta']]

In [4]:
columnas = ['superficie','lat', 'lon','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

In [5]:
set_entrenamiento = propiedades.loc[(propiedades.Year >= 2016) &((propiedades.Year < 2017) | (propiedades.Month < 6))\
                                    ,columnas_precio]
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),columnas_precio].head(20000)

set_entrenamiento_datos = set_entrenamiento.loc[:,columnas]
set_entrenamiento_resultado = set_entrenamiento.loc[:,'price_aprox_usd']

In [6]:
tree = DecisionTreeRegressor()
tree.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply(lambda x: tree.predict(x)[0],axis = 1)
precision = tree.score(set_pruebas.loc[:,columnas],set_pruebas.price_aprox_usd) * 100
error = mean_squared_error(set_pruebas.price_aprox_usd,set_pruebas.resultado)
print("Precision = {:.2f} % , error = {}".format(precision, error))

Precision = 98.28 % , error = 1989833789.7358894


In [7]:
res = []

In [8]:
lista_criterio = ['mse','friedman_mse']
lista_max_depth = [None,1,5,10]
lista_min_samples_split = [2,3]
lista_min_samples_leaf = [1,10]
lista_features = [0.4,0.6,1.0]

for criterio in lista_criterio:
    for max_depth in lista_max_depth:
        for min_sample in lista_min_samples_split:
            for min_leaf in lista_min_samples_leaf:
                for feature in lista_features:
                    
                    tree = DecisionTreeRegressor(criterion = criterio, max_depth = max_depth, min_samples_split = min_sample,\
                                    min_samples_leaf = min_leaf, max_features = feature)
                    tree.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
                    set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply\
                            (lambda x: tree.predict(x)[0],axis = 1)
                    precision = tree.score(set_pruebas.loc[:,columnas],set_pruebas.loc[:,'price_aprox_usd']) * 100
                    error = mean_squared_error(set_pruebas.price_aprox_usd,set_pruebas.resultado)
                    res.append((criterio,max_depth,min_sample,min_leaf,feature,precision,error))
                    print(criterio,' - ',max_depth,' - ',min_sample,' - ',min_leaf,' - ',feature)

mse  -  None  -  2  -  1  -  0.4
mse  -  None  -  2  -  1  -  0.6
mse  -  None  -  2  -  1  -  1.0
mse  -  None  -  2  -  10  -  0.4
mse  -  None  -  2  -  10  -  0.6
mse  -  None  -  2  -  10  -  1.0
mse  -  None  -  3  -  1  -  0.4
mse  -  None  -  3  -  1  -  0.6
mse  -  None  -  3  -  1  -  1.0
mse  -  None  -  3  -  10  -  0.4
mse  -  None  -  3  -  10  -  0.6
mse  -  None  -  3  -  10  -  1.0
mse  -  1  -  2  -  1  -  0.4
mse  -  1  -  2  -  1  -  0.6
mse  -  1  -  2  -  1  -  1.0
mse  -  1  -  2  -  10  -  0.4
mse  -  1  -  2  -  10  -  0.6
mse  -  1  -  2  -  10  -  1.0
mse  -  1  -  3  -  1  -  0.4
mse  -  1  -  3  -  1  -  0.6
mse  -  1  -  3  -  1  -  1.0
mse  -  1  -  3  -  10  -  0.4
mse  -  1  -  3  -  10  -  0.6
mse  -  1  -  3  -  10  -  1.0
mse  -  5  -  2  -  1  -  0.4
mse  -  5  -  2  -  1  -  0.6
mse  -  5  -  2  -  1  -  1.0
mse  -  5  -  2  -  10  -  0.4
mse  -  5  -  2  -  10  -  0.6
mse  -  5  -  2  -  10  -  1.0
mse  -  5  -  3  -  1  -  0.4
mse  -  5  -  3  - 

In [9]:
for r in res:
    print("Criterio = {}, max_depth = {}, min_sample_split = {}, min_sample_leaf = {}, features = {}, precision = {:.2f} % , error = {}"\
          .format(r[0],r[1],r[2],r[3],r[4],r[5],r[6]))

Criterio = mse, max_depth = None, min_sample_split = 2, min_sample_leaf = 1, features = 0.4, precision = 98.05 % , error = 2255960804.956864
Criterio = mse, max_depth = None, min_sample_split = 2, min_sample_leaf = 1, features = 0.6, precision = 98.28 % , error = 1989025537.2791593
Criterio = mse, max_depth = None, min_sample_split = 2, min_sample_leaf = 1, features = 1.0, precision = 98.21 % , error = 2074825073.4398386
Criterio = mse, max_depth = None, min_sample_split = 2, min_sample_leaf = 10, features = 0.4, precision = 64.17 % , error = 41481380227.237434
Criterio = mse, max_depth = None, min_sample_split = 2, min_sample_leaf = 10, features = 0.6, precision = 57.61 % , error = 49080302558.42927
Criterio = mse, max_depth = None, min_sample_split = 2, min_sample_leaf = 10, features = 1.0, precision = 63.76 % , error = 41963364067.33917
Criterio = mse, max_depth = None, min_sample_split = 3, min_sample_leaf = 1, features = 0.4, precision = 95.81 % , error = 4850759665.253196
Criteri

In [10]:
min_error = float('inf')
max_precision = 0
tupla_min_error = ()
tupla_max_precision = ()
for r in res:
    if r[6] < min_error:
        min_error = r[6]
        tupla_min_error = r
    if r[5] > max_precision:
        max_precision = r[5]
        tupla_max_precision = r
        
print("Mayor precision = Criterio = {}, max_depth = {}, min_sample_split = {}, min_sample_leaf = {}, features = {}, precision = {:.2f} % , error = {}".\
              format(tupla_max_precision[0],tupla_max_precision[1],tupla_max_precision[2],tupla_max_precision[3],tupla_max_precision[4],tupla_max_precision[5],tupla_max_precision[6]))
print("Menor error = Criterio = {}, max_depth = {}, min_sample_split = {}, min_sample_leaf = {}, features = {}, precision = {:.2f} % , error = {}".\
              format(tupla_min_error[0],tupla_min_error[1],tupla_min_error[2],tupla_min_error[3],tupla_min_error[4],tupla_min_error[5],tupla_min_error[6]))

Mayor precision = Criterio = mse, max_depth = None, min_sample_split = 2, min_sample_leaf = 1, features = 0.6, precision = 98.28 % , error = 1989025537.2791593
Menor error = Criterio = mse, max_depth = None, min_sample_split = 2, min_sample_leaf = 1, features = 0.6, precision = 98.28 % , error = 1989025537.2791593


## Tomamos criterio = mse, max_depth = None ,min_sample = 2 ,min_leaf = 1, features = 0.6

In [49]:
#Probamos distintas combinaciones de columnas, esta es la que dio mejores resultados

columnas = ['superficie','lat', 'lon','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

In [50]:
set_entrenamiento = propiedades.loc[(propiedades.Year >= 2016) &((propiedades.Year < 2017) | (propiedades.Month < 6))\
                                    ,columnas_precio]
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),columnas_precio].head(20000)

set_entrenamiento_datos = set_entrenamiento.loc[:,columnas]
set_entrenamiento_resultado = set_entrenamiento.loc[:,'price_aprox_usd']

In [51]:
criterio = 'mse'
max_depth = None
min_sample = 2
min_leaf = 1
feature = 0.6

In [52]:
tree = DecisionTreeRegressor(criterion = criterio, max_depth = max_depth, min_samples_split = min_sample,\
                                    min_samples_leaf = min_leaf, max_features = feature)
tree.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply(lambda x: tree.predict(x)[0],axis = 1)
precision = tree.score(set_pruebas.loc[:,columnas],set_pruebas.price_aprox_usd) * 100
error = mean_squared_error(set_pruebas.price_aprox_usd,set_pruebas.resultado)
print("Precision = {:.2f} % , error = {}".format(precision, error))

Precision = 98.10 % , error = 2198804392.142215


# calculo con los verdaderos datos a analizar

In [53]:
analizar = pd.read_csv("/home/mati/Desktop/properati_dataset_modificado.csv")

In [54]:
analizar.loc[:,'price_usd'] = analizar.loc[:,columnas].apply(lambda x: tree.predict(x)[0],axis = 1)

In [55]:
resultado = analizar.loc[:,['id','price_usd']]

In [56]:
resultado.to_csv('resultados/Decision_tree_2', index = False)

# Usando Grid Search y Cross Validation

In [2]:
propiedades = pd.read_csv('/home/mati/Desktop/set_datos_propiedades.csv')

propiedades = propiedades.loc[(propiedades.price_aprox_usd.notnull()) & (propiedades.superficie.notnull())\
                              & (propiedades.lat.notnull()) & (propiedades.lon.notnull()) & (propiedades.Year >=2016)]

columnas = ['superficie','lat', 'lon','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

set_entrenamiento_datos = propiedades.loc[:,columnas]
set_entrenamiento_resultado = propiedades.loc[:,'price_aprox_usd']

In [3]:
# Parametros

criterios = ['mse','friedman_mse']
max_depth = [None,1,5,10]
samples_split = [2,3]
features = [0.4,0.6,1.0]

parametros = {"criterion" : criterios, "max_depth" : max_depth, \
              "min_samples_split" : samples_split, "max_features" : features}

iteraciones_cross_validation = 75
arbol = DecisionTreeRegressor()

In [4]:
inicio = time.strftime("%X")

grid = GridSearchCV( estimator = arbol, param_grid = parametros, n_jobs = -1, cv = iteraciones_cross_validation)

grid.fit(set_entrenamiento_datos, set_entrenamiento_resultado)

score = grid.best_score_ * 100
mejores_parametros = grid.best_params_
fin = time.strftime("%X")

print("Tiempo: {} --- {} \n Precision: {:.2f} \n Parametros = {}".format(inicio,fin,score,mejores_parametros))

Tiempo: 17:47:24 --- 19:17:30 
 Precision: 95.69 
 Parametros = {'criterion': 'mse', 'max_depth': None, 'max_features': 0.4, 'min_samples_split': 3}


In [5]:
#Con 10 iteraciones de CV: Precision: 92.52       12:17:11 --- 12:29:12
#               Parametros = {'criterion': 'mse', 'max_depth': None, 'max_features': 0.6, 'min_samples_split': 3}

#Con 30 iteraciones de CV:  Precision: 94.97       11:24:03 --- 12:00:09
#              Parametros = {'criterion': 'mse', 'max_depth': None, 'max_features': 0.4, 'min_samples_split': 2}

#Con 50 iteraciones de CV:    Precision: 95.61     12:09:55 --- 13:12:40 
#              Parametros = {'criterion': 'mse', 'max_depth': None, 'max_features': 0.6, 'min_samples_split': 2}

In [6]:
analizar = pd.read_csv("/home/mati/Desktop/properati_dataset_modificado.csv")

analizar.loc[:,'price_usd'] = analizar.loc[:,columnas].apply(lambda x: grid.predict(x)[0],axis = 1)

resultado = analizar.loc[:,['id','price_usd']]

resultado.to_csv('resultados/Decision_tree_GridSearch.csv', index = False)

In [7]:
joblib.dump(grid, 'algoritmos/decision_tree.pkl')

['algoritmos/decision_tree.pkl']

## Hago bagging con el mejor algoritmo

In [2]:
propiedades = pd.read_csv('/home/mati/Desktop/set_datos_propiedades.csv')

propiedades = propiedades.loc[(propiedades.price_aprox_usd.notnull()) & (propiedades.superficie.notnull())\
                              & (propiedades.lat.notnull()) & (propiedades.lon.notnull()) & (propiedades.Year >=2016)]

columnas = ['superficie','lat', 'lon','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

set_pruebas = analizar = pd.read_csv("/home/mati/Desktop/properati_dataset_modificado.csv")
set_pruebas.loc[:,'price_usd'] = 0.0

In [3]:
cant = 10

for i in range(cant):
    datos = propiedades.sample(frac = 0.4)
    set_entrenamiento_datos = datos.loc[:,columnas]
    set_entrenamiento_resultado = datos.loc[:,'price_aprox_usd']
    
    tree = DecisionTreeRegressor(criterion = 'mse', max_depth = None, max_features = 0.4, min_samples_split = 3)
    
    tree.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
    set_pruebas.loc[:,'res'] = set_pruebas.loc[:,columnas].apply(lambda x: tree.predict(x)[0],axis = 1)
    set_pruebas.loc[:,'price_usd'] = set_pruebas.loc[:,'price_usd'] + set_pruebas.loc[:,'res']

In [4]:
set_pruebas.loc[:,'price_usd'] = set_pruebas.loc[:,'price_usd'] / cant

resultado = set_pruebas.loc[:,['id','price_usd']]

resultado.to_csv('resultados/Decision_tree_Cross_Validation.csv', index = False)

# Ahora pruebo Grid Search con menos datos 
#### Elimino aquellos con precio muy bajo o muy alto

In [3]:
propiedades = pd.read_csv('/home/mati/Desktop/set_datos_propiedades.csv')

In [4]:
sup_min = 10
sup_max = 150

lat_min = -35
lat_max = -34

lon_min = -59
lon_max = -58

precio_min = 5000
precio_max = 1000000

In [5]:
propiedades = propiedades.loc[(propiedades.price_aprox_usd.notnull()) & (propiedades.superficie.notnull())\
                              & (propiedades.lat.notnull()) & (propiedades.lon.notnull()) & (propiedades.Year >=2016)\
                        & (propiedades.price_aprox_usd <= precio_max) & (propiedades.price_aprox_usd >= precio_min) &\
                         (propiedades.superficie <= sup_max) & (propiedades.superficie >= sup_min) &\
                      (propiedades.lat <= lat_max) & (propiedades.lat >= lat_min) &\
                       (propiedades.lon <= lon_max) & (propiedades.lon >= lon_min),:]

columnas = ['superficie','lat', 'lon','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

set_entrenamiento_datos = propiedades.loc[:,columnas]
set_entrenamiento_resultado = propiedades.loc[:,'price_aprox_usd']

In [10]:
# Parametros

criterios = ['mse','friedman_mse']
max_depth = [None,50,75]
features = [0.4,0.6,0.8,1.0]

parametros = {"criterion" : criterios, "max_depth" : max_depth, "max_features" : features}

iteraciones_cross_validation = 12
arbol = DecisionTreeRegressor()

In [11]:
inicio = time.strftime("%X")

grid = GridSearchCV( estimator = arbol, param_grid = parametros, n_jobs = -1, cv = iteraciones_cross_validation)

grid.fit(set_entrenamiento_datos, set_entrenamiento_resultado)

score = grid.best_score_ * 100
mejores_parametros = grid.best_params_
fin = time.strftime("%X")

print("Tiempo: {} --- {} \n Precision: {:.2f} \n Parametros = {}".format(inicio,fin,score,mejores_parametros))

Tiempo: 14:27:27 --- 14:34:39 
 Precision: 92.42 
 Parametros = {'criterion': 'friedman_mse', 'max_depth': 75, 'max_features': 0.8}


In [12]:
analizar = pd.read_csv("/home/mati/Desktop/properati_dataset_modificado.csv")

analizar.loc[:,'price_usd'] = analizar.loc[:,columnas].apply(lambda x: grid.predict(x)[0],axis = 1)

resultado = analizar.loc[:,['id','price_usd']]

resultado.to_csv('resultados/Decision_tree_GridSearch_menosDatos.csv', index = False)

In [13]:
joblib.dump(grid, 'algoritmos/decision_tree_2.pkl')

['algoritmos/decision_tree_2.pkl']

## Hago bagging con el mejor algoritmo

In [14]:
propiedades = pd.read_csv('/home/mati/Desktop/set_datos_propiedades.csv')

sup_min = 10
sup_max = 150

lat_min = -35
lat_max = -34

lon_min = -59
lon_max = -58

precio_min = 5000
precio_max = 1000000

propiedades = propiedades.loc[(propiedades.price_aprox_usd.notnull()) & (propiedades.superficie.notnull())\
                              & (propiedades.lat.notnull()) & (propiedades.lon.notnull()) & (propiedades.Year >=2016)\
                        & (propiedades.price_aprox_usd <= precio_max) & (propiedades.price_aprox_usd >= precio_min) &\
                         (propiedades.superficie <= sup_max) & (propiedades.superficie >= sup_min) &\
                      (propiedades.lat <= lat_max) & (propiedades.lat >= lat_min) &\
                       (propiedades.lon <= lon_max) & (propiedades.lon >= lon_min),:]

columnas = ['superficie','lat', 'lon','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

set_pruebas = analizar = pd.read_csv("/home/mati/Desktop/properati_dataset_modificado.csv")
set_pruebas.loc[:,'price_usd'] = 0.0

In [15]:
cant = 10

for i in range(cant):
    datos = propiedades.sample(frac = 0.7)
    set_entrenamiento_datos = datos.loc[:,columnas]
    set_entrenamiento_resultado = datos.loc[:,'price_aprox_usd']
    
    tree = DecisionTreeRegressor(criterion = 'friedman_mse', max_depth = 75, max_features = 0.8)
    
    tree.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
    set_pruebas.loc[:,'res'] = set_pruebas.loc[:,columnas].apply(lambda x: tree.predict(x)[0],axis = 1)
    set_pruebas.loc[:,'price_usd'] = set_pruebas.loc[:,'price_usd'] + set_pruebas.loc[:,'res']

In [16]:
set_pruebas.loc[:,'price_usd'] = set_pruebas.loc[:,'price_usd'] / cant

resultado = set_pruebas.loc[:,['id','price_usd']]

resultado.to_csv('resultados/Decision_tree_Cross_Validation_menos_datos.csv', index = False)

# Usando Grid Search y Cross Validation con las fechas

In [3]:
propiedades = pd.read_csv('/home/mati/Desktop/set_datos_propiedades.csv')

propiedades = propiedades.loc[(propiedades.price_aprox_usd.notnull()) & (propiedades.superficie.notnull())\
                              & (propiedades.lat.notnull()) & (propiedades.lon.notnull()) & (propiedades.Year >=2016)]

propiedades.loc[:,'fecha'] = propiedades.loc[:,['Year', 'Month']].apply(lambda x: int(str(x[0]) + str(x[1])), axis = 1)

columnas = ['superficie','lat', 'lon','property_type_encoded','seguridad','gimnasio', 'aire','pileta', 'cochera', 'fecha']
columnas_precio = columnas + ['price_aprox_usd']

set_entrenamiento_datos = propiedades.loc[:,columnas]
set_entrenamiento_resultado = propiedades.loc[:,'price_aprox_usd']

In [14]:
# Parametros

criterios = ['mse','friedman_mse']
max_depth = [None,5,10]
features = [0.4,0.6,1.0]

parametros = {"criterion" : criterios, "max_depth" : max_depth, "max_features" : features}

iteraciones_cross_validation = 25
arbol = DecisionTreeRegressor()

In [15]:
inicio = time.strftime("%X")

grid = GridSearchCV( estimator = arbol, param_grid = parametros, n_jobs = 2, cv = iteraciones_cross_validation)

grid.fit(set_entrenamiento_datos, set_entrenamiento_resultado)

score = grid.best_score_ * 100
mejores_parametros = grid.best_params_
fin = time.strftime("%X")

print("Tiempo: {} --- {} \n Precision: {:.2f} \n Parametros = {}".format(inicio,fin,score,mejores_parametros))

Tiempo: 20:55:33 --- 21:14:42 
 Precision: 94.61 
 Parametros = {'criterion': 'friedman_mse', 'max_depth': None, 'max_features': 0.6}


In [17]:
analizar = pd.read_csv("/home/mati/Desktop/properati_dataset_modificado.csv")

analizar.loc[:,'Month'] = analizar.created_on.apply(lambda x: int(x.split('-')[1]))
analizar.loc[:,'Year'] = analizar.created_on.apply(lambda x: int(x.split('-')[0]))
analizar.loc[:,'fecha'] = analizar.loc[:,['Year', 'Month']].apply(lambda x: int(str(x[0]) + str(x[1])), axis = 1)

analizar.loc[:,'price_usd'] = analizar.loc[:,columnas].apply(lambda x: grid.predict(x)[0],axis = 1)

resultado = analizar.loc[:,['id','price_usd']]

resultado.to_csv('resultados/Decision_tree_GridSearch_fechas.csv', index = False)

In [18]:
joblib.dump(grid, 'algoritmos/decision_tree_3.pkl')

['algoritmos/decision_tree_3.pkl']

## Hago bagging con el mejor algoritmo

In [19]:
analizar.loc[:,'price_usd'] = 0.0

In [20]:
cant = 10

for i in range(cant):
    datos = propiedades.sample(frac = 0.4)
    set_entrenamiento_datos = datos.loc[:,columnas]
    set_entrenamiento_resultado = datos.loc[:,'price_aprox_usd']
    
    tree = DecisionTreeRegressor(criterion = 'friedman_mse', max_depth = None, max_features = 0.6)
    
    tree.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
    analizar.loc[:,'res'] = analizar.loc[:,columnas].apply(lambda x: tree.predict(x)[0],axis = 1)
    analizar.loc[:,'price_usd'] = analizar.loc[:,'price_usd'] + analizar.loc[:,'res']

In [21]:
analizar.loc[:,'price_usd'] = analizar.loc[:,'price_usd'] / cant

resultado = analizar.loc[:,['id','price_usd']]

resultado.to_csv('resultados/Decision_tree_Cross_Validation_fechas.csv', index = False)